In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd


from sklearn.model_selection import train_test_split as tts

from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.ensemble import RandomForestClassifier as RFC

from sklearn.model_selection import GridSearchCV        # por fuerza bruta
from sklearn.model_selection import RandomizedSearchCV

salaries=pd.read_csv('../data/salaries_data.csv')
testeo=pd.read_csv('../data/testeo.csv')

salaries.drop(columns=['salary_currency', 'salary'], inplace=True)  

testeo.drop(columns=['salary_currency', 'salary'], inplace=True) 

total=pd.concat([salaries, testeo])

print(total.shape)

total=pd.get_dummies(total, drop_first=True)

total.shape

total.fillna(0)

salaries=total.iloc[:500,:]
testeo=total.iloc[500:,:]

testeo.drop(columns='salary_in_usd', inplace=True)



(607, 9)


In [2]:
def grid(modelo, param):
    
    g=GridSearchCV(modelo, # modelo de sklearn
                   param,  # dictio de parametros
                   cv=5,   # nº de cortes del cross-validation
                   return_train_score=True, # error en entrenamiento para checkear
                   n_jobs=-1  # usa todos los nucleos disponibles
                  )

    g.fit(X_train, y_train)

    print('Acierto test: {:.2f}'.format(g.score(X_test, y_test)))
    print('Acierto train: {:.2f}'.format(g.score(X_train, y_train)))
    print('Mejores parametros: {}'.format(g.best_params_))
    print('Mejor acierto cv: {:.2f}'.format(g.best_score_))

    return g.best_estimator_.fit(X_train, y_train)  # mejor modelo encontrado ya entrenado

In [3]:
X=salaries.drop(columns=['salary_in_usd'])
y=salaries.salary_in_usd
X_train, X_test, y_train, y_test=tts(X, y, random_state=42)

In [4]:
param={'n_estimators': [10, 50, 100, 150, 200, 500],
       'max_depth': [1, 5, 10, 15, 20]}

modelo=grid(RFR(), param)

Acierto test: 0.45
Acierto train: 0.81
Mejores parametros: {'max_depth': 10, 'n_estimators': 150}
Mejor acierto cv: 0.47


In [5]:
rfr=RFR(n_estimators=150,max_depth=10)
rfr.fit(X_train, y_train)
pred_rfr=rfr.predict(X)

In [7]:
from sklearn.metrics import mean_squared_error as mse 
print('rfr',mse( y, pred_rfr, squared=False))

rfr 35256.61897808026


In [8]:
pred_rfr=rfr.predict(testeo)
pred_rfr=pd.DataFrame(pred_rfr, columns=['salary_in_usd'])
pred_rfr.to_csv('../data/rfr3.csv',index_label='id')

In [ ]:
from sklearn.linear_model import Ridge 

param={'alpha': [1.0, 2.0, 3.0, 4.0, 5.0, 6.0],
       'normalize': [True,False],
       'max_iter':[100,250,500,1000,1500,2000,2500]}
